# django 불러오기

In [1]:
import os, sys
# PWD = os.getenv('PWD')
PWD = os.path.abspath('.')

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

In [2]:
PWD

'c:\\Users\\user\\mydjango_board'

In [3]:
init_django('mydjango_board') #project name

# model import

In [4]:
from board.models import Board, Comment

# Insert or Update

In [5]:
board = Board(
    title="게시글1",
    content="내용1",
)

In [6]:
board.save()

In [7]:
board

<Board: Board object (1)>

-> 내용 확인

#### Manager 객체 - objects

In [8]:
board = Board.objects.create(
    title="게시글2",
    content="내용2"
)

In [9]:
board

<Board: Board object (2)>

In [10]:
board = Board.objects.create(
    title="게시글3",
    content="내용3"
)

In [11]:
board

<Board: Board object (3)>

### 여러 게시글을 한번에 insert하기

In [12]:
data = []
for i in range(10):
    board = Board(title=f"새로운 게시글{i}", content=f"여러게시글 insert {i}")
    data.append(board)
    


In [13]:
print(data)

[<Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>, <Board: Board object (None)>]


In [14]:
Board.objects.bulk_create(data)

[<Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>,
 <Board: Board object (None)>]

In [15]:
board = Board(title="게시물 추가", content="내용")
board

<Board: Board object (None)>

In [16]:
board.save()

# 조회하기

## raw query로 조회하기

In [17]:
result = Board.objects.raw('SELECT * FROM board_board')
result

<RawQuerySet: SELECT * FROM board_board>

In [18]:
for i in result[:3]:
    print(i)

Board object (1)
Board object (2)
Board object (3)


## orm으로 조회하기

### 전체 조회하기

In [19]:
board_list = Board.objects.all()

In [20]:
print(board_list.query)

SELECT `board_board`.`id`, `board_board`.`title`, `board_board`.`content`, `board_board`.`created_at`, `board_board`.`updated_at` FROM `board_board`


### filter 걸어서 조회하기(where)

In [21]:
board_list = Board.objects.filter(title="게시물 추가")
board_list

<QuerySet [<Board: Board object (14)>]>

In [22]:
board_list = Board.objects.order_by('-created_at') # order by desc
board_list

<QuerySet [<Board: Board object (14)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (3)>, <Board: Board object (2)>, <Board: Board object (1)>]>

In [23]:
board_list = Board.objects.order_by('-created_at')[2:4] # order by desc limit(2) offset(2)
board_list

<QuerySet [<Board: Board object (12)>, <Board: Board object (11)>]>

###  filter not 표시

In [24]:
board_list = Board.objects.exclude(title="게시물 추가")
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>]>

In [25]:
from django.db.models import Q

0001  
1001  


-> 
and: 0001  
or: 1001  
not 0001 => 1110  
xor: 1000  

####  비트연산
- ~: not
- &: and
- |: or
- ^: xor

In [26]:
board_list = Board.objects.filter(~Q(title="게시물 추가"))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>]>

## 조회하기 고급

### 다양한 조건

#### or

In [27]:
board_list = Board.objects.filter(Q(title="게시글1") | Q(title="게시글2"))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>]>

##### and 와 연산자

- gte: greater than equal
- lte: less than equal
- lt: less than
- gt: greater than

In [28]:
board_list = Board.objects.filter(Q(title__endswith="글1") & Q(id__gte=3))
board_list

<QuerySet [<Board: Board object (5)>]>

In [29]:
board_list = Board.objects.filter(Q(title__startswith="게시") & Q(created_at__year=2023))
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (14)>]>

# 관계 추가하기

1번 댓글에 댓글을 3가지 이상 입력하세요

In [10]:
comment = Comment(
    board_id = 1,
    content = "댓글 내용2"
)
comment.save()

In [11]:
board = Board.objects.get(id=1)
comment = Comment.objects.create(
    board=board,
    content = "댓글 내용3"
)

In [12]:
for i in range(10):
    Comment.objects.create(
        board = Board.objects.get(id=2),
        content = f"댓글{i}"
    )

## 관계 조회하기

In [5]:
Comment.objects.all()

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>, <Comment: Comment object (3)>, <Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>, <Comment: Comment object (7)>, <Comment: Comment object (8)>, <Comment: Comment object (9)>, <Comment: Comment object (10)>, <Comment: Comment object (11)>, <Comment: Comment object (12)>, <Comment: Comment object (13)>]>

In [6]:
board_list = Board.objects.filter(id__lte=3)
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>]>

In [7]:
board = board_list[0]
board

<Board: Board object (1)>

###  board가 가지고 있는 comment 모두 조회

In [8]:
board.comment_set.all()

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>]>

In [10]:
board_list = Board.objects.all()
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>, <Board: Board object (6)>, <Board: Board object (7)>, <Board: Board object (8)>, <Board: Board object (9)>, <Board: Board object (10)>, <Board: Board object (11)>, <Board: Board object (12)>, <Board: Board object (13)>, <Board: Board object (14)>, <Board: Board object (15)>]>

In [13]:
di = {}
for i in board_list:
    di[i.id] = len(i.comment_set.all())

In [16]:
for i in range(10):
    print(di[i])

AttributeError: 'dict' object has no attribute 'i'

In [9]:
for board in board_list:
    print(board)
    print(board.comment_set.all())
    print('-'*10)

Board object (1)
<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>]>
----------
Board object (2)
<QuerySet [<Comment: Comment object (3)>, <Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>, <Comment: Comment object (7)>, <Comment: Comment object (8)>, <Comment: Comment object (9)>, <Comment: Comment object (10)>, <Comment: Comment object (11)>, <Comment: Comment object (12)>]>
----------
Board object (3)
<QuerySet []>
----------


In [18]:
board_list = Board.objects.filter(id__lte=5).prefetch_related('comment_set')
board_list

<QuerySet [<Board: Board object (1)>, <Board: Board object (2)>, <Board: Board object (3)>, <Board: Board object (4)>, <Board: Board object (5)>]>

In [19]:
comment_list = board_list[0].comment_set.all()
comment_list

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>]>

In [20]:
comment = comment_list[0]
comment.board

<Board: Board object (1)>

# transaction

In [21]:
from django.db import transaction

In [22]:
@transaction.atomic
def sample():
    board = Board.objects.create(
        title='게시글',
        content = "atomic 게시글"
    )
    Comment.objects.create(
        board=board,
        content="atomic 댓글"
    )

In [23]:
sample()

In [24]:
with transaction.atomic():
    pass

In [25]:
file = open("sample.txt", 'wt')
file.write("파일 쓰기")
file.close()

In [26]:
with open("sample.txt", 'wt') as f:
    f.write("파일 쓰기")

### decorator - @ = 함수다

In [27]:
def new_decorator(func):
    def wrap():
        print("decorator 시작")
        func()
        print("decorator 종료")
    return wrap

In [28]:
@new_decorator
def my_func():
    print('my_func')
my_func()

decorator 시작
my_func
decorator 종료


In [29]:
def my_func2():
    print('my_func')

In [30]:
new_decorator(my_func2)()

decorator 시작
my_func
decorator 종료
